# VARIMA Baseline (Vector Auto Regressive Integrated Moving Average) Baseline



In [2]:

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.varmax import VARMAX
import itertools
import warnings


In [6]:
# ----------------------------
# Load and prepare data
# ----------------------------
df = pd.read_parquet("/storage/bln-aq/data/2024-citsci-pm2.5-daily.parquet")
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(df.columns)

# Pivot to wide form: columns = sensors
df_wide = df.pivot(index='timestamp', columns='item_id', values='target').ffill()



# ----------------------------
# VARIMA hyperparameter selection via AIC/BIC
# ----------------------------
p_values = range(1, 4)  # lag order
d_values = range(0, 2)  # differencing

best_aic = float("inf")
best_bic = float("inf")
best_order_aic = None
best_order_bic = None
warnings.filterwarnings("ignore")


Index(['timestamp', 'item_id', 'target'], dtype='object')


In [ ]:
for p, d in itertools.product(p_values, d_values):
    try:
        model = VARMAX(df_wide, order=(p,d), trend='c')
        fit = model.fit(disp=False)
        if fit.aic < best_aic:
            best_aic = fit.aic
            best_order_aic = (p,d)
        if fit.bic < best_bic:
            best_bic = fit.bic
            best_order_bic = (p,d)
    except:
        continue

print(f"Best AIC={best_aic:.2f}, order={best_order_aic}")
print(f"Best BIC={best_bic:.2f}, order={best_order_bic}")

# ----------------------------
# Fit final VARIMA model (choose one, e.g., AIC)
# ----------------------------
p,d = best_order_aic
model = VARMAX(df_wide, order=(p,d), trend='c')
fit = model.fit(disp=False)

# ----------------------------
# Forecast 1 day ahead
# ----------------------------
forecast = fit.get_forecast(steps=3)
pred_mean = forecast.predicted_mean

# Estimate simple quantiles from residuals
residuals = df_wide - fit.fittedvalues
q_lower = pred_mean + np.percentile(residuals, 10, axis=0)
q_upper = pred_mean + np.percentile(residuals, 90, axis=0)

# Combine results
pred_df = pd.DataFrame({
    'timestamp': [df_wide.index[-1] + pd.Timedelta(days=1)],
})
for col in df_wide.columns:
    pred_df[f'sensor_{col}_median'] = pred_mean[col].values
    pred_df[f'sensor_{col}_p10'] = q_lower[col].values
    pred_df[f'sensor_{col}_p90'] = q_upper[col].values

print(pred_df)
pred_df.to_csv("/storage/bln-aq/data/varima_daily_forecast.csv", index=False)